In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
assets = dicts.set_index('Asset')
print(assets)

market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
market = pd.concat([market, greenbull], axis=1).ffill().bfill()
print(market.columns)


              Market Currency     Forex           Class  PriceFmt AmountFmt  \
Asset                                                                         
EUR         EUREUR=X           EUREUR=X         Deposit                       
€           EUREUR=X      EUR  EUREUR=X            Cash  {:.0f} €    {:.0f}   
$              EUR=X      EUR  EUREUR=X            Cash  {:.4f} €    {:.0f}   
USDT           EUR=X     EUR   EUREUR=X            Cash  {:.4f} €    {:.0f}   
ESE           ESE.PA        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
PUST                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
RS2K                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
OBLI         OBLI.PA        €  EUREUR=X    Fixed Income  {:.2f} €    {:.0f}   
GOLD         GOLD.PA        €  EUREUR=X     Commodities  {:.2f} €    {:.0f}   
BTC          BTC-USD     USDT  EURUSD=X  Cryptocurrency  {:.0f} $    {:.4f}   
ETH                      USDT  EURUSD=X  Cryptocurre

In [11]:

df = pd.DataFrame(index=market.index, columns=pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]]))#, names=['portfolio', 'class', 'asset', 'metric']))
df.head()

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{classs}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)

        df[portfolio, classs, asset, 'Quantity'] = dfa['Quantity']
        df[portfolio, classs, asset, 'Operation'] = dfa['Operation']
        df[portfolio, classs, asset, 'Cotation'] = market[assets.loc[asset]['Market']]
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        classs = assets.loc[currency]['Class']
        print(f"\t\t{classs}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)
        
        if (portfolio, classs, currency, 'Quantity') not in df.columns:
            df[portfolio, classs, currency, 'Quantity'] = 0
        if (portfolio, classs, currency, 'Operation') not in df.columns:
            df[portfolio, classs, currency, 'Operation'] = 0
        if (portfolio, classs, currency, 'Cotation') not in df.columns:
            df[portfolio, classs, currency, 'Cotation'] = market[assets.loc[currency]['Market']]

        df[portfolio, classs, currency, 'Quantity'] -= dfc['Operation']
        df[portfolio, classs, currency, 'Operation'] -= dfc['Operation'] * df[portfolio, classs, currency, 'Cotation']


    print(f"\tBuild datatable ...")
    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{asset}")

        position, invested = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            position[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Quantity'])
            invested[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Operation'])

        df = pd.concat([df, pd.DataFrame({
                (portfolio, classs, asset, 'Position'): position,
                (portfolio, classs, asset, 'Invested'): invested,
            })], axis=1)

        df[portfolio, classs, asset, 'PRU'] = df[portfolio, classs, asset, 'Invested'] / df[portfolio, classs, asset, 'Position']
        df[portfolio, classs, asset, 'Value'] = df[portfolio, classs, asset, 'Position'] * df[portfolio, classs, asset, 'Cotation']
        df[portfolio, classs, asset, 'PnL'] = df[portfolio, classs, asset, 'Value'] - df[portfolio, classs, asset, 'Invested']
            
        toEUR = market[assets.loc[asset]['Forex']]
        df[portfolio, classs, asset, 'InvestedEUR'] = df[portfolio, classs, asset, 'Invested'] / toEUR
        df[portfolio, classs, asset, 'ValueEUR'] = df[portfolio, classs, asset, 'Value'] / toEUR
        df[portfolio, classs, asset, 'PnLEUR'] = df[portfolio, classs, asset, 'PnL'] / toEUR


    print(f"\tConcate class data ...")
    for classs in np.unique(df[portfolio].columns.get_level_values(0)):
        print(f"\t\t{classs}")
        df[portfolio, classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'InvestedEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'ValueEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'PnLEUR']].sum(axis=1)

    # df[portfolio, 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'InvestedEUR']].sum(axis=1)
    # df[portfolio, 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'ValueEUR']].sum(axis=1)
    # df[portfolio, 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'PnLEUR']].sum(axis=1)

    print(f"... OK")

for classs in np.unique(df.columns.get_level_values(1)):
    df['All', classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'InvestedEUR']].sum(axis=1)
    df['All', classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'ValueEUR']].sum(axis=1)
    df['All', classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'PnLEUR']].sum(axis=1)

# df['All', 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'InvestedEUR']].sum(axis=1)
# df['All', 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'ValueEUR']].sum(axis=1)
# df['All', 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'PnLEUR']].sum(axis=1)


for column in df.columns:
    print(column)


DMA
	Assets
		Cash:	€
	Currency
		Deposit:	EUR
	Build datatable ...
		EUR
		€
	Concate class data ...
		Cash
		Deposit
... OK
ZEN
	Assets
		Cash:	$
		Cryptocurrency:	BTC
		Equities:	ESE
		Commodities:	GOLD
		Equities:	GREENBULL
		Fixed Income:	OBLI
		Cash:	USDT
		Cash:	€
	Currency
		Cash:	$
		Deposit:	EUR
		Cash:	USDT
		Cash:	€
	Build datatable ...
		$
		BTC
		ESE
		EUR
		GOLD
		GREENBULL
		OBLI
		USDT
		€
	Concate class data ...
		Cash
		Commodities
		Cryptocurrency
		Deposit
		Equities
		Fixed Income
... OK
('DMA', 'Cash', '€', 'Quantity')
('DMA', 'Cash', '€', 'Operation')
('DMA', 'Cash', '€', 'Cotation')
('DMA', 'Deposit', 'EUR', 'Quantity')
('DMA', 'Deposit', 'EUR', 'Operation')
('DMA', 'Deposit', 'EUR', 'Cotation')
('DMA', 'Deposit', 'EUR', 'Position')
('DMA', 'Deposit', 'EUR', 'Invested')
('DMA', 'Deposit', 'EUR', 'PRU')
('DMA', 'Deposit', 'EUR', 'Value')
('DMA', 'Deposit', 'EUR', 'PnL')
('DMA', 'Deposit', 'EUR', 'InvestedEUR')
('DMA', 'Deposit', 'EUR', 'ValueEUR')
('DMA', 'Depos

In [14]:
dates = ["2021-04-30","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]
df.loc[dates, pd.IndexSlice['All', :, 'All', :]]

All                                                     \
                    Cash                           Commodities                
                     All                                   All                
             InvestedEUR      ValueEUR      PnLEUR InvestedEUR     ValueEUR   
2021-04-30      6.889211      0.000000   -6.889211        0.00     0.000000   
2021-07-30     15.003087     10.930803   -4.072284        0.00     0.000000   
2021-08-10    415.003087    411.074700   -3.928387        0.00     0.000000   
2021-10-01    418.220347    411.231610   -6.988738        0.00     0.000000   
2021-11-01    420.672848    411.248900   -9.423948        0.00     0.000000   
2021-12-01    891.118850    874.833418  -16.285432        0.00     0.000000   
2021-12-31    449.618842    433.554001  -16.064841        0.00     0.000000   
2022-02-01    455.391337    438.291500  -17.099837        0.00     0.000000   
2022-03-24  10055.391337  10039.068510  -16.322827        0.00     0.000000   
2022-04-01  12076.721825  12042.022200  -34.699625        0.00     0.000000   
2022-04-05  11595.292022  11578.544723  -16.747298      280.99   280.855988   
2022-04-13  11173.020217  11181.989528    8.969311      500.05   507.570015   
2022-05-02  12683.221617  12738.517128   55.295511      781.04   776.820030   
2022-04-19  13173.020217  13188.909283   15.889066      500.05   505.050011   
2022-06-01  12230.156326  12251.155593   20.999268     1054.03  1035.419998   
2022-07-01  12020.186122  12073.128230   52.942107     1054.03  1036.499977   
2022-07-04  11743.186122  11800.852239   57.666117     1331.03  1310.524971   
2022-07-05  13243.186122  13300.934881   57.748758     1331.03  1304.046051   
2022-08-05  12819.015721  12897.243025   78.227304     1540.52  1533.752075   
2022-08-25  13319.015721  13423.459136  104.443415     1540.52  1542.881927   
2022-10-12  11831.310463  11962.479267  131.168803     1815.19  1779.440063   
2023-02-10  10827.310463  10884.355827   57.045364     1815.19  1800.239944   

                                                                           \
                      Cryptocurrency                              Deposit   
                                 All                                  All   
               PnLEUR    InvestedEUR     ValueEUR      PnLEUR InvestedEUR   
2021-04-30   0.000000       0.000000     0.000000    0.000000      -500.0   
2021-07-30   0.000000       0.000000     0.000000    0.000000     -1000.0   
2021-08-10   0.000000       0.000000     0.000000    0.000000     -1400.0   
2021-10-01   0.000000       0.000000     0.000000    0.000000     -1900.0   
2021-11-01   0.000000       0.000000     0.000000    0.000000     -2400.0   
2021-12-01   0.000000       0.000000     0.000000    0.000000     -3400.0   
2021-12-31   0.000000       0.000000     0.000000    0.000000     -3400.0   
2022-02-01   0.000000       0.000000     0.000000    0.000000     -3900.0   
2022-03-24   0.000000       0.000000     0.000000    0.000000    -13500.0   
2022-04-01   0.000000       0.000000     0.000000    0.000000    -16000.0   
2022-04-05  -0.134012     200.439803   199.226925   -1.212877    -16000.0   
2022-04-13   7.520015     406.423598   387.858030  -18.565568    -16000.0   
2022-05-02  -4.219970     626.425773   577.795566  -48.630207    -18000.0   
2022-04-19   5.000011     408.095591   392.632525  -15.463067    -18000.0   
2022-06-01 -18.610002     819.816828   632.953580 -186.863248    -18500.0   
2022-07-01 -17.530023    1049.850972   629.703603 -420.147369    -18500.0   
2022-07-04 -20.505029    1054.250985   663.908234 -390.342752    -18500.0   
2022-07-05 -26.983949    1054.327958   662.606347 -391.721611    -20000.0   
2022-08-05  -6.767925    1288.082349   994.043023 -294.039326    -20000.0   
2022-08-25   2.361927    1324.092029   947.752527 -376.339501    -20500.0   
2022-10-12 -35.749937    1587.436461  1090.968016 -496.468445    -20500.0   
2023-02-10 -14.950056    1433.437056  111

In [26]:
s = df.iloc[-1]
s.loc[pd.IndexSlice['ZEN', ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum()

11631.6020149702

In [43]:
rows = {}
for ptf in ['ZEN']:
    rows[ptf] = {
        'Value': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum(),
        'Invested': s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum(),
        'Cash': s.loc[pd.IndexSlice[ptf, 'Cash', 'All', 'ValueEUR']],
        'PnL': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'PnLEUR']].sum(),
        'Deposited': -s.loc[pd.IndexSlice[ptf, 'Deposit', 'All', 'InvestedEUR']],
    }
df1 = pd.DataFrame(rows).transpose()

print(f"PnL (value-invested) : {'OK' if df1['Value'].values[0] - df1['Invested'].values[0] - df1['Cash'].values[0] == df1['PnL'].values[0] else 'error'}")

df1

PnL (value-invested) : error


,Cash,Deposited,Invested,PnL,Value
ZEN,2387.691071,12000.0,9949.147338,-644.855787,11631.602015
